In [1]:
# import libraries
import pandas as pd
import numpy as np
import json
import nltk
import re
import string
import os
from tqdm import tqdm
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from IPython.display import JSON

from sklearn.feature_extraction.text import TfidfVectorizer

# import own functions
import preprocessing
import data_access

In [2]:
# filepath = 'data/cord-19/metadata.csv'
# df = preprocessing.clean_metadata_for_lda(filepath)

In [3]:
# df.to_csv('data/processed/metadata_clean_lda.csv', sep='\t')

In [4]:
df = preprocessing.load_cleaned_metadata('data/processed/metadata_clean_lda.csv')

In [5]:
df['pdf_json_files'] = df['pdf_json_files'].fillna('nan')
df['pmc_json_files'] = df['pmc_json_files'].fillna('nan')

In [7]:
def first(s):
    if ";" in s:
        s = s.split(';')[0].strip()
    return s

dest_directory = 'data/cord-19/body_text/lda_raw/'

for i in tqdm(range(len(df))):
    row = df.iloc[i]
    
    # get title and abstract
    title_abstract = row.title + '\n\n' + row.abstract + '\n\n'

    # get body text, if there's a file for it
    if row.pdf_json_files != 'nan':
        path = 'data/cord-19/' + first(row.pdf_json_files)
        body = data_access.get_body_text_from_path(path)
    elif row.pmc_json_files != 'nan':
        path = 'data/cord-19/' + first(row.pmc_json_files)
        body = data_access.get_body_text_from_path(path)
    else:
        body = ''
    
    # write text to file
    uid = row.cord_uid
    dest_path = dest_directory + uid + '.txt'
    with open(dest_path, 'w') as file:
        file.write(title_abstract)
        file.write(body)

100%|██████████| 437966/437966 [19:26<00:00, 375.38it/s] 


In [8]:
len(df)

437966

In [9]:
len(os.listdir('data/cord-19/body_text/lda_raw/'))

437967